In [29]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import log_loss, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

In [30]:
# reading data
raw_data = pd.read_excel("data200rows.xlsx")

In [31]:
raw_data

,date_dmy,departure_time,departure_place,depature_latitude,depature_longitude,destination_time,destination_place,destination_latitude,destination_longitude
0,2021-01-04,08:00:44,home,60.194619,24.898739,08:16:26,office,60.197538,24.927277
1,2021-01-04,13:00:45,office,60.197538,24.927277,13:05:42,subway,60.198592,24.932554
2,2021-01-04,13:25:22,subway,60.198592,24.932554,13:32:51,office,60.197538,24.927277
3,2021-01-04,18:40:42,office,60.197538,24.927277,18:55:14,restaurant,60.190062,24.917299
4,2021-01-04,19:50:14,restaurant,60.190062,24.917299,20:05:11,home,60.194619,24.898739
...,...,...,...,...,...,...,...,...,...
195,2021-03-06,08:03:42,home,60.194619,24.898739,09:20:52,golf,60.261921,24.684964
196,2021-03-06,18:32:02,golf,60.261921,24.684964,19:40:52,home,60.194619,24.898739
197,2021-03-06,09:10:15,home,60.194619,24.898739,10:02:36,Sports complex,60.177833,24.781436
198,2021-03-06,15:25:00,Sports complex,60.177833,24.781436,15:38:59,restaurant,60.171078,24.778268


In [32]:
raw_data['day'] = 0

def findDay(date):
    daynum = datetime.strptime(str(date),'%Y-%m-%d %H:%M:%S').weekday() 
    #days =["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return daynum

for i,d in enumerate(raw_data['date_dmy']):
  day = findDay(d)
  raw_data['day'][i] = day     

In [33]:
raw_data

,date_dmy,departure_time,departure_place,depature_latitude,depature_longitude,destination_time,destination_place,destination_latitude,destination_longitude,day
0,2021-01-04,08:00:44,home,60.194619,24.898739,08:16:26,office,60.197538,24.927277,0
1,2021-01-04,13:00:45,office,60.197538,24.927277,13:05:42,subway,60.198592,24.932554,0
2,2021-01-04,13:25:22,subway,60.198592,24.932554,13:32:51,office,60.197538,24.927277,0
3,2021-01-04,18:40:42,office,60.197538,24.927277,18:55:14,restaurant,60.190062,24.917299,0
4,2021-01-04,19:50:14,restaurant,60.190062,24.917299,20:05:11,home,60.194619,24.898739,0
...,...,...,...,...,...,...,...,...,...,...
195,2021-03-06,08:03:42,home,60.194619,24.898739,09:20:52,golf,60.261921,24.684964,5
196,2021-03-06,18:32:02,golf,60.261921,24.684964,19:40:52,home,60.194619,24.898739,5
197,2021-03-06,09:10:15,home,60.194619,24.898739,10:02:36,Sports complex,60.177833,24.781436,5
198,2021-03-06,15:25:00,Sports complex,60.177833,24.781436,15:38:59,restaurant,60.171078,24.778268,5


In [34]:
raw_data['hour'] = 0
raw_data['minutes'] = 0
raw_data['seconds'] = 0

def HMS(time):
  d = datetime.strptime(str(time), '%H:%M:%S')
  return d
  
for i,t in enumerate(raw_data['departure_time']):
  ti = HMS(t)
  raw_data['hour'][i] = int(ti.strftime("%H")) 
  raw_data['minutes'][i] = int(ti.strftime("%M")) 
  raw_data['seconds'][i] = int(ti.strftime("%S"))  

In [35]:
raw_data.head()

,date_dmy,departure_time,departure_place,depature_latitude,depature_longitude,destination_time,destination_place,destination_latitude,destination_longitude,day,hour,minutes,seconds
0,2021-01-04,08:00:44,home,60.194619,24.898739,08:16:26,office,60.197538,24.927277,0,8,0,44
1,2021-01-04,13:00:45,office,60.197538,24.927277,13:05:42,subway,60.198592,24.932554,0,13,0,45
2,2021-01-04,13:25:22,subway,60.198592,24.932554,13:32:51,office,60.197538,24.927277,0,13,25,22
3,2021-01-04,18:40:42,office,60.197538,24.927277,18:55:14,restaurant,60.190062,24.917299,0,18,40,42
4,2021-01-04,19:50:14,restaurant,60.190062,24.917299,20:05:11,home,60.194619,24.898739,0,19,50,14


In [36]:
raw_data.isnull().sum()

date_dmy                 0
departure_time           0
departure_place          0
depature_latitude        0
depature_longitude       0
destination_time         0
destination_place        0
destination_latitude     0
destination_longitude    0
day                      0
hour                     0
minutes                  0
seconds                  0
dtype: int64

In [37]:
len(raw_data['destination_place'].unique())

33

In [38]:
Places = {}

In [39]:
Places

{}

In [40]:
count = 0
for place in raw_data['departure_place']:
    if place in Places:
        continue
    else:
        Places[place] = count
        count=count+1
for place in raw_data['destination_place']:
    if place in Places:
        continue
    else:
        Places[place] = count
        count=count+1

In [41]:
Places

{'home': 0,
 'office': 1,
 'subway': 2,
 'restaurant': 3,
 'bar': 4,
 'boating': 5,
 'excurtion': 6,
 'cafe': 7,
 'hospital': 8,
 'store': 9,
 'holyplace': 10,
 'bank': 11,
 'library': 12,
 'swimming beach': 13,
 'chaging center': 14,
 'park': 15,
 'rolercoster': 16,
 'grocery': 17,
 'stadium': 18,
 'pharamacy': 19,
 'golf': 20,
 'bicycle shop': 21,
 'parking': 22,
 'airport': 23,
 'cemetery': 24,
 'zoo': 25,
 'salon': 26,
 'pub ': 27,
 'Pharmacy': 28,
 'Convention center': 29,
 'University': 30,
 'Sports complex': 31,
 'museum': 32}

In [42]:
raw_data['encoded_departure'] = 0
raw_data['encoded_destination'] = 0

for i,pl in enumerate(raw_data['departure_place']):
    raw_data['encoded_departure'][i] = Places[pl]
for i,pl in enumerate(raw_data['destination_place']):
    raw_data['encoded_destination'][i] = Places[pl]  

In [43]:
raw_data

,date_dmy,departure_time,departure_place,depature_latitude,depature_longitude,destination_time,destination_place,destination_latitude,destination_longitude,day,hour,minutes,seconds,encoded_departure,encoded_destination
0,2021-01-04,08:00:44,home,60.194619,24.898739,08:16:26,office,60.197538,24.927277,0,8,0,44,0,1
1,2021-01-04,13:00:45,office,60.197538,24.927277,13:05:42,subway,60.198592,24.932554,0,13,0,45,1,2
2,2021-01-04,13:25:22,subway,60.198592,24.932554,13:32:51,office,60.197538,24.927277,0,13,25,22,2,1
3,2021-01-04,18:40:42,office,60.197538,24.927277,18:55:14,restaurant,60.190062,24.917299,0,18,40,42,1,3
4,2021-01-04,19:50:14,restaurant,60.190062,24.917299,20:05:11,home,60.194619,24.898739,0,19,50,14,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2021-03-06,08:03:42,home,60.194619,24.898739,09:20:52,golf,60.261921,24.684964,5,8,3,42,0,20
196,2021-03-06,18:32:02,golf,60.261921,24.684964,19:40:52,home,60.194619,24.898739,5,18,32,2,20,0
197,2021-03-06,09:10:15,home,60.194619,24.898739,10:02:36,Sports complex,60.177833,24.781436,5,9,10,15,0,31
198,2021-03-06,15:25:00,Sports complex,60.177833,24.781436,15:38:59,restaurant,60.171078,24.778268,5,15,25,0,31,3


In [44]:
X = raw_data[['day', 'hour','minutes','seconds', 'encoded_departure']]
y = raw_data['destination_place']

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create kNN classifier
knn = KNeighborsClassifier()  # Fit the classifier to the data
knn.fit(X_train, y_train)

#kNN accuracy and other measures
y_pred = knn.predict(X_test)
print("k Nearest Neighbors model accuracy(in %):", knn.score(X_test, y_test)*100)

# Classification Report for kNN
predictions = knn.predict(X_test)
classi_report = classification_report(y_test, predictions)
print(classi_report)

k Nearest Neighbors model accuracy(in %): 35.0
                   precision    recall  f1-score   support

Convention center       0.00      0.00      0.00         0
         Pharmacy       0.00      0.00      0.00         1
          airport       0.00      0.00      0.00         2
              bar       0.00      0.00      0.00         1
             cafe       0.00      0.00      0.00         1
             golf       0.00      0.00      0.00         1
          grocery       0.00      0.00      0.00         1
        holyplace       0.00      0.00      0.00         3
             home       0.48      1.00      0.65        12
           office       0.20      0.18      0.19        11
        pharamacy       0.00      0.00      0.00         1
             pub        0.00      0.00      0.00         1
       restaurant       0.00      0.00      0.00         4
          stadium       0.00      0.00      0.00         1

         accuracy                           0.35        40
       

In [45]:
from sklearn.tree import DecisionTreeClassifier

# Create DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

print("Decision Tree model accuracy(in %):", clf.score(X_test, y_test)*100)

# Classification Report for kNN
predictions = clf.predict(X_test)
classi_report = classification_report(y_test, predictions)
print(classi_report)

Decision Tree model accuracy(in %): 42.5
                   precision    recall  f1-score   support

Convention center       0.00      0.00      0.00         0
         Pharmacy       0.00      0.00      0.00         1
          airport       0.00      0.00      0.00         2
              bar       1.00      1.00      1.00         1
     bicycle shop       0.00      0.00      0.00         0
          boating       0.00      0.00      0.00         0
             cafe       0.00      0.00      0.00         1
         cemetery       0.00      0.00      0.00         0
             golf       0.00      0.00      0.00         1
          grocery       0.00      0.00      0.00         1
        holyplace       0.00      0.00      0.00         3
             home       0.71      0.83      0.77        12
         hospital       0.00      0.00      0.00         0
           office       0.44      0.36      0.40        11
             park       0.00      0.00      0.00         0
        pharam